# ETL Clean Kaggle Data

## Deliverable 3: Extract and Transform Kaggle Data

In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [2]:
#  Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):

    
    
    

    return movie

In [3]:
# 1 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def etl_movie_data(wiki_data, kaggle_data, ratings_data):
    ##################
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_data = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
    ratings_data = pd.read_csv(f'{file_dir}ratings.csv')    

    ##################
    # Open and read the Wikipedia data JSON file.
    with open(f'{file_dir}wikipedia-movies.json', mode='r') as file:
        wiki_movies_raw = json.load(file)    
    
    ##################
    # Write a list comprehension to filter out TV shows.
    wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]
    ##################
    # Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_movies = [clean_movie(movie) for movie in wiki_movies]    

    ##################
    # Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(clean_movies)

    ##################
    # Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        print(len(wiki_movies_df))
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
        print(len(wiki_movies_df))
        wiki_movies_df.head()        
    except Exception as e:
        print(f"Exception: {e}")
        print("Occurred while processing imdb IDs on the following movie data:")
        print(wiki_movies_df['imdb_id'])

    ##################
    # Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    # Trim down the dataset by removing any columns with >= 90% Null Values
    wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]    

    ##################
    # Create a variable that will hold the non-null values from the “Box office” column.
    box_office = wiki_movies_df['Box office'].dropna()
    
    ##################
    # Convert the box office data created in Step 8 to string values using the lambda and join functions.
    def is_not_a_string(x):
        return type(x) != str
    
    box_office[box_office.map(is_not_a_string)]
    box_office[box_office.map(lambda x: type(x) != str)]
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)    

    ##################
    # Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'
   
    ##################
    # Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illi?on)'    

    ##################
    # Add the parse_dollars function.
    def parse_dollars(s):
        # if s is not a string, return NaN
        if type(s) != str:
            return np.nan

        # if input is of the form $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

            # remove dollar sign and " million"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

            # convert to float and multiply by a million
            value = float(s) * 10**6

            # return value
            return value

        # if input is of the form $###.# billion
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

            # remove dollar sign and " billion"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

            # convert to float and multiply by a billion
            value = float(s) * 10**9

            # return value
            return value

        # if input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

            # remove dollar sign and commas
            s = re.sub('\$|,','', s)

            # convert to float
            value = float(s)

            # return value
            return value

        # otherwise, return NaN
        else:
            return np.nan
        
    ##################
    # Clean the box office column in the wiki_movies_df DataFrame.
    wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

    # We no longer need the original Box office column, so drop it:
    wiki_movies_df.drop('Box office', axis=1, inplace=True)
    
    ##################
    # Clean the budget column in the wiki_movies_df DataFrame.

    # Create a budget variable:
    budget = wiki_movies_df['Budget'].dropna()

    # Convert any lists to strings:
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)

    # Remove any values between a dollar sign and any hyphens or dashes (for budgets given in ranges):
    budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
    
    # Use form_one and form_two to parse the budget data the same way as was done for box office:
    matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE, na=False)
    matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE, na=False)
    
    # Remove the citation references with the following:
    budget = budget.str.replace(r'\[\d+\]\s*', '')

    # Parse the budget values in the DataFrame:
    wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

    # We no longer need the original Budget column, so drop it:
    wiki_movies_df.drop('Budget', axis=1, inplace=True)

    ##################
    # Clean the release date column in the wiki_movies_df DataFrame.
    
    # Create a variable to hold non-null values of Release date in a DF, converting lists to strings:
    release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

    # Create variables to hold the 4 different date forms:
    date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]?\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[0123]\d'
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}'
    
    # Extract the dates and add the release_date column to the data frame:
    wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)

    ##################
    # Clean the running time column in the wiki_movies_df DataFrame.

    # Create a variable to hold non-null values of Running time in a DF, converting lists to strings:
    running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

    # Extract the running times with the new format to include different forms of hours and minutes:
    running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')

    # Convert the strings to be numeric values:
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    
    # Convert hour and minute capture groups to minutes if pure mintutes group is zero and save in DF:
    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
    
    # Drop the Running time column from the DF since it is no longer needed.
    wiki_movies_df.drop('Running time', axis=1, inplace=True)
     
    ##################
    # 2. Clean the Kaggle metadata.

    # Keep rows where the adult column is False, and then drop the adult column.
    kaggle_data = kaggle_data[kaggle_data['adult'] == 'False'].drop('adult',axis='columns')

    # Convert the string values to boolean:
    kaggle_data['video'] = kaggle_data['video'] == 'True'
    
    # Convert columns that should be numeric to only contain numeric values:
    kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
    kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
    kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')

    # Convert release_date values from string to datetime
    kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])

    # Convert ratings data timestamp to datetime using unix origin and time unit as seconds
    # and assign it to the timestamp column:
    ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

    ##################
    # 3. Merged the two DataFrames into the movies DataFrame.


    ##################
    # 4. Drop unnecessary columns from the merged DataFrame.


    ##################
    # 5. Add in the function to fill in the missing Kaggle data.


    ##################
    # 6. Call the function in Step 5 with the DataFrame and columns as the arguments.


    ##################
    # 7. Filter the movies DataFrame for specific columns.


    ##################
    # 8. Rename the columns in the movies DataFrame.


    ##################
    # 9. Transform and merge the ratings DataFrame.
    
    return wiki_movies_df, movies_with_ratings_df, movies_df


In [4]:
# 10. Create the path to your file directory and variables for the three files.
file_dir = '../Resources'
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia_movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [5]:
# 11. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load()

In [6]:
# 12. Set the DataFrames from the return statement equal to the file names in Step 11. 
wiki_movies_df = wiki_file
movies_with_ratings_df = kaggle_file
movies_df = ratings_file

In [7]:
# 13. Check the wiki_movies_df DataFrame. 
wiki_movies_df.head()

In [8]:
# 14. Check the movies_with_ratings_df DataFrame.
movies_with_ratings_df.head()

In [9]:
# 15. Check the movies_df DataFrame. 
movies_df.head()